### Processes vs Threads

In [ ]:
def do_something(thread_number):
  print(f'Working {thread_number} ...')
  time.sleep(0.15)
  print(f'Thread {thread_number} was finished.')

In [ ]:
# Python GIL (Global Interpreter Lock): apenas um thread é executado por vez.
# Dessa forma, o desempenho do processo single-threaded e do processo multi-threaded será o mesmo em python e isso se deve ao GIL.
# Para contornar essa limitação, é preciso recorrer ao Jython, Cython, Iron Phyton, ou usar processos, ao invés de threads, visto que
# para processos não há essa limitação.
# Um detalhe importante: quando você faz a cópia de um processo (fork), além das instruções, 
# o pool de dados em memória é também copiado para o novo processo (não compartilham).
# Com threads é diferente, pois quando você cria threads elas compartilham o espaço de memória, ou seja, os dados são compartilhados entre elas.
# Essas diferenças tornam o uso de processos mais exigentes em memória.
import time
from threading import Thread

for i in range(5):
  t = Thread(target=do_something, args=(i,))
  t.start()

Working 0 ...
Working 1 ...
Working 2 ...Working 3 ...
Working 4 ...

Thread 0 was finished.
Thread 1 was finished.
Thread 3 was finished.
Thread 2 was finished.
Thread 4 was finished.


Ha 3 formas de iniciar um processo em Python: spawn (apenas Linux), fork e forkserver.
A diferença entre eles está na forma que a cópia do espaço de memória é feita.

- Fork: é uma simples cópia do espaço de memória (Default no Unix). 

Fork()-ing the parent processes and continuing with the same processes image in both parent and child. This method is fast, but potentially unreliable when parent state is complex

- Spawn: não copia tudo. Apenas o que for necessário. Isso permite economizar um pouco de memória. É um pouco mais lento para iniciar. (Default no Windows e MacOS)

- Forkserver: é um modo intermediário que tenta salvar um pouco de memória e sendo um pouco mais rápido. Quando o processo é iniciado é feito um forkserver que fica salvo em uma região da memória. A partir dessa região, são feitos forks. Então, apenas a memória é clonada.
When the program starts and selects the forkserver start method, a server process is started. 
From then on, whenever a new process is needed, the parent process connects to the server and requests that it fork a new process. Available on Unix platforms.
It consists of a separate Python server with that has a relatively simple state and which is fork()-ed when a new processes is needed. This method combines the speed of Fork()-ing with good reliability (because the parent being forked is in a simple state).
If you want something to be inherited by child processes from the parent, this must be specified in the forkserver state.
https://stackoverflow.com/questions/63424251/multiprocessing-in-python-what-gets-inherited-by-forkserver-process-from-paren

In [ ]:
import multiprocessing
from multiprocessing import Process

In [ ]:
if __name__ == '__main__':
  multiprocessing.set_start_method('spawn')
  for i in range(5):
    p = Process(target=do_something, args=(i,))
    p.start()

### IPC - Interprocess Communication

Can be done through messages and memory sharing.

### Memory Sharing

In [53]:
# No exemplo, a seguir, vamos testar o compartilhamento de memória entre threads.
import json
import urllib.request
import time

# global variable
finished_count = 0

def count_letters(url, frequency_table, thread_num, count=True):
  print(f'counting letters {thread_num}...')
  response = urllib.request.urlopen(url)
  content = str(response.read())
  for item in content:
    if item.lower() in frequency_table:
      frequency_table[item.lower()] += 1
  if count:
    global finished_count
    finished_count += 1

def main():
  frequency_table = {}
  for letter in 'abcdefghijklmnopqrstuvxzwy':
    frequency_table[letter] = 0

  start = time.time()
  for i in range(1000, 1020):
    count_letters(f'http://www.rfc-editor.org/rfc/rfc{i}.txt', frequency_table, i, False)
  end = time.time()

  print(json.dumps(frequency_table, indent=4), end - start)

main()

counting letters 1000...
counting letters 1001...
counting letters 1002...
counting letters 1003...
counting letters 1004...
counting letters 1005...
counting letters 1006...
counting letters 1007...
counting letters 1008...
counting letters 1009...
counting letters 1010...
counting letters 1011...
counting letters 1012...
counting letters 1013...
counting letters 1014...
counting letters 1015...
counting letters 1016...
counting letters 1017...
counting letters 1018...
counting letters 1019...
{
    "a": 80014,
    "b": 16998,
    "c": 48003,
    "d": 40501,
    "e": 140093,
    "f": 26074,
    "g": 19010,
    "h": 36316,
    "i": 79913,
    "j": 2170,
    "k": 6614,
    "l": 38305,
    "m": 31176,
    "n": 135371,
    "o": 84258,
    "p": 32270,
    "q": 2835,
    "r": 75326,
    "s": 79790,
    "t": 103557,
    "u": 27572,
    "v": 10580,
    "x": 4719,
    "z": 1115,
    "w": 14195,
    "y": 13914
} 21.472853183746338


In [55]:
# with thread
# Quando as threads compartilham a memória e escrevem em conjunto um mesmo recurso, podem ocorrer inconsistências nos resultados.
# É preciso implementar sincronização entre elas para que isso não ocorra.
# Veja como o tempo foi expressivamente menor usando threads!

from threading import Thread

def main_with_thread():
  frequency_table = {}

  for letter in 'abcdefghijklmnopqrstuvxzwy':
    frequency_table[letter] = 0

  start = time.time()
  for i in range(1000, 1020):
    t = Thread(target=count_letters, args=(f'http://www.rfc-editor.org/rfc/rfc{i}.txt', frequency_table, i)).start()
  
  # wait all threads to finish
  print('finished_count', finished_count)
  while finished_count < 20:
    print('finished_count', finished_count)
    time.sleep(0.5)

  end = time.time()

  time.sleep(1)
  print(json.dumps(frequency_table, indent=4), end - start)

main_with_thread()


counting letters 1000...
counting letters 1001...
counting letters 1002...
counting letters 1003...
counting letters 1004...
counting letters 1005...
counting letters 1006...counting letters 1007...
counting letters 1008...

counting letters 1009...
counting letters 1010...
counting letters 1011...
counting letters 1012...
counting letters 1013...counting letters 1014...

counting letters 1015...counting letters 1016...

counting letters 1017...
counting letters 1018...
counting letters 1019...
finished_count 1
finished_count 1
finished_count 1
finished_count 7
finished_count 19
{
    "a": 80014,
    "b": 16998,
    "c": 47553,
    "d": 39345,
    "e": 138429,
    "f": 26074,
    "g": 19010,
    "h": 36316,
    "i": 77493,
    "j": 2170,
    "k": 6614,
    "l": 37257,
    "m": 31176,
    "n": 129616,
    "o": 82299,
    "p": 32270,
    "q": 2835,
    "r": 73181,
    "s": 78021,
    "t": 103557,
    "u": 27572,
    "v": 10580,
    "x": 4719,
    "z": 1115,
    "w": 14195,
    "y": 13914

##### Sincronização de Threads